In [186]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from collections import OrderedDict
import torch.nn as nn
import pandas as pd
import torch

### Sequential with module and forward

In [187]:
def NeuralNetwork(Features, hidden, target):

	# construct a sequential neural network
	mlpModel = nn.Sequential(OrderedDict([
		("hidden_layer", nn.Linear(Features, hidden)),
		("activation_function", nn.Sigmoid()),
		("output_layer", nn.Linear(hidden, target)),
		("out", nn.Sigmoid())                                       # we use this to get values between 0 and 1
	]))
	
	return mlpModel

### Create Data

In [188]:
data = pd.read_csv('data.csv', header=None)
data

,0,1,2
0,0.78051,-0.063669,1
1,0.28774,0.291390,1
2,0.40714,0.178780,1
3,0.29230,0.421700,1
4,0.50922,0.352560,1
...,...,...,...
95,0.77029,0.701400,0
96,0.73156,0.717820,0
97,0.44556,0.579910,0
98,0.85275,0.859870,0


### Create X and y and split

In [189]:
# create x and y with tensor
X = torch.tensor(data.drop(2, axis=1).values, dtype= torch.float)
y = torch.tensor(data[2].values, dtype= torch.float).reshape(-1, 1)

In [190]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

In [191]:
X_train.shape

torch.Size([70, 2])

In [192]:
y_test.shape

torch.Size([30, 1])

### Train model

In [193]:
torch.manual_seed(0)
model = NeuralNetwork(X.shape[1], 10, 1)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.BCELoss()

In [194]:
trainloss = []                   
epochs = 500

# pass the data through the model for a number of epochs
for epoch in range(epochs):

    # put model in training model
    model.train()
    # forward pass on train data using the forward() method inside
    y_pred = model(X_train)
    # calculate the loss
    loss = loss_fn(y_pred, y_train)
    # zero the gradients of the optimizer
    optimizer.zero_grad()
    # perform backpropagation on the loss
    loss.backward()
    # progress the optimizer
    optimizer.step()

    # store loss 
    trainloss.append(loss.detach().item())
    print(f'Epoch {epochs}, train loss {loss.item():.2f}')

Epoch 500, train loss 0.69
Epoch 500, train loss 0.69
Epoch 500, train loss 0.69
Epoch 500, train loss 0.68
Epoch 500, train loss 0.68
Epoch 500, train loss 0.68
Epoch 500, train loss 0.68
Epoch 500, train loss 0.68
Epoch 500, train loss 0.68
Epoch 500, train loss 0.68
Epoch 500, train loss 0.67
Epoch 500, train loss 0.67
Epoch 500, train loss 0.67
Epoch 500, train loss 0.67
Epoch 500, train loss 0.67
Epoch 500, train loss 0.67
Epoch 500, train loss 0.66
Epoch 500, train loss 0.66
Epoch 500, train loss 0.66
Epoch 500, train loss 0.66
Epoch 500, train loss 0.66
Epoch 500, train loss 0.65
Epoch 500, train loss 0.65
Epoch 500, train loss 0.65
Epoch 500, train loss 0.65
Epoch 500, train loss 0.65
Epoch 500, train loss 0.64
Epoch 500, train loss 0.64
Epoch 500, train loss 0.64
Epoch 500, train loss 0.64
Epoch 500, train loss 0.64
Epoch 500, train loss 0.63
Epoch 500, train loss 0.63
Epoch 500, train loss 0.63
Epoch 500, train loss 0.63
Epoch 500, train loss 0.62
Epoch 500, train loss 0.62
E

### Test model

In [195]:
model.eval()

Sequential(
  (hidden_layer): Linear(in_features=2, out_features=10, bias=True)
  (activation_function): Sigmoid()
  (output_layer): Linear(in_features=10, out_features=1, bias=True)
  (out): Sigmoid()
)

In [196]:
test_losses = []
test_acc = []

with torch.no_grad():
            test_preds = model.forward(X_test)
            test_loss = loss_fn(test_preds, y_test)
            test_losses.append(test_loss.item())

            # to get prediction values between 0 and 1
            out_values = test_preds > 0.5

            # to check test accuracy
            #accuracy = (out_values == y_test).sum() / out_values.shape[0]
            accuracy = (out_values == y_test).sum() / len(y_test)
            test_acc.append(accuracy)
print(f'Epoch: {epochs + 1}, train loss: {loss.item():.2f}, test loss: {test_loss.item():.2f}, accuracy: {test_acc}')


Epoch: 501, train loss: 0.18, test loss: 0.05, accuracy: [tensor(0.9667)]


In [202]:
trainloss = []
test_losses = []
test_acc = []
epochs = 500


for epoch in range(0, epochs):
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    trainloss.append(loss.detach().item())

    model.eval()
    with torch.no_grad():
            test_preds = model.forward(X_test)
            test_loss = loss_fn(test_preds, y_test)
            test_losses.append(test_loss.item())
            out_values = test_preds > 0.5
            accuracy = (out_values == y_test).sum() / len(y_test)
            test_acc.append(accuracy.item())
    model.train()
#print(f'Epoch: {epoch + 1}, train loss: {trainloss:.2f}, test loss: {test_losses:.2f}, accuracy: {test_acc:.2f}')
#print(trainloss)
# display model progress on the current training batch
trainTemplate = "epoch: {} train loss: {:.3f} train accuracy: {:.3f}"
print(trainTemplate.format(epoch + 1,(trainloss), (test_acc))) 
    

TypeError: unsupported format string passed to list.__format__